In [36]:
import numpy as np              # Para operaciones matematicas
import pandas as pd             # Manejo de datos / dataframes

In [2]:
import requests
from bs4 import BeautifulSoup

In [153]:
import requests
from bs4 import BeautifulSoup

URL = 'https://articulo.mercadolibre.com.ec/MEC-430304751-camara-de-5-metros-endoscopio-infaroja-para-celular-android-_JM#c_id=/home/category/deal-items/element&c_uid=4a58e1e7-9a10-488d-bb44-2f8ebe33ccdd'
page = requests.get(URL)

soup = BeautifulSoup(page.content, 'html.parser')


##### Datos generales

In [184]:
NombreArticulo = soup.find_all("h1", {"class": "ui-pdp-title"})[0].contents[0]
NombreArticulo= NombreArticulo.replace("/"," ")
NombreArticulo

'Camilla Portátil Para Masajes Reforzada Con Maleta'

In [155]:
Image_articulo = soup.find_all("figure", {"class": "ui-pdp-gallery__figure"})[0]
Image_articulo= Image_articulo.find_all("img")[0]['data-zoom']
Image_articulo

'https://http2.mlstatic.com/D_NQ_NP_811579-MEC45598429009_042021-F.jpg'

In [156]:
precio = soup.find_all("span", {"class": "price-tag-fraction"})[0].contents[0]
precio

'99'

##### Try - para 2 casos

##### caso 1

In [157]:
url = soup.find_all("a", {"class": "ui-pdp-action-modal__link"})[-1]['href']
url

'https://articulo.mercadolibre.com.ec/noindex/profile/feedbacks?noIndex=true&itemId=MEC430304751&controlled=true&modal=true&rating=all'

In [158]:
# fase 2.1        
page_user_middleware = requests.get(url)
soup_user_middleware = BeautifulSoup(page_user_middleware.content, 'html.parser')

In [159]:
new_url = soup_user_middleware.find_all("p", {"class": "feedback-profile-link"})[0]
new_url

<p class="feedback-profile-link">
<a href="http://perfil.mercadolibre.com.ec/CUENCA.DISTRIBUIDORA" target="_blank">
			Ver todas las calificaciones del vendedor
		</a>
</p>

In [160]:
url = new_url.find_all("a")[0]
url = url['href']

###### Caso 2

In [50]:
url = soup.find_all("a", {"class": "ui-pdp-media__action ui-box-component__action"})[0]['href']
url

[]

In [105]:
url

'http://perfil.mercadolibre.com.ec/RAMSA+IMPORTACIONES'

### otra fase

In [161]:
page_user = requests.get(url)
soup_user = BeautifulSoup(page_user.content, 'html.parser')

In [183]:
name_vendedor = soup_user.find_all("h3", {"class": "store-info__name"})[0].contents[0]
name_vendedor

'CUENCA.DISTRIBUIDORA'

In [163]:
Califications = soup_user.find_all("span", {"class": "buyers-feedback-qualification"})

Califications

[<span class="buyers-feedback-qualification" id="feedback_good">Positiva<!-- --> (<!-- -->3251<!-- -->)</span>,
 <span class="buyers-feedback-qualification" id="feedback_good">Neutral<!-- --> (<!-- -->579<!-- -->)</span>,
 <span class="buyers-feedback-qualification" id="feedback_good">Negativa<!-- --> (<!-- -->401<!-- -->)</span>]

In [164]:
calification_points = []
for C in Califications:
    calification_points.append(C.contents[4])
calification_points

['3251', '579', '401']

In [165]:
reputation = soup_user.find_all("div", {"class": "data-level__wrapper"})[0]
reputation= reputation.find_all("span", {"class": "data-level__number"})
recomendado = reputation[0].contents[0]
ventas_completadas = reputation[1].contents[0]
años_vendiendo = reputation[2].contents[0]

In [166]:
time = soup_user.find_all("p", {"class": "data-level__description"})[-1]
time = time.find_all("span")[0].contents[-1].split(' ')[1]
time

'años'

In [178]:
NombreArticulo='laptop hp'

In [185]:
new_url = 'https://listado.mercadolibre.com.ec/'+NombreArticulo
new_url

'https://listado.mercadolibre.com.ec/Camilla Portátil Para Masajes Reforzada Con Maleta'

In [186]:
page_other_article = requests.get(new_url)
soup_other_article = BeautifulSoup(page_other_article.content, 'html.parser') 

In [205]:
card = soup_other_article.find_all("img", {"class": "ui-search-result-image__element"})
len(card)

23

In [232]:
datos = []
sume = 0
maxi = 0
mini = 9999999

for item in card:
    # obtener precios
    price = soup_other_article.find_all("a", {"title": item['alt']})
    #print(price[1].find_all("span", {"class": "price-tag-fraction"}))
    try:
        num = float(price[1].find_all("span", {"class": "price-tag-fraction"})[0].contents[0])        
    except:
        search = soup_other_article.find_all("div", {"class": "andes-card andes-card--flat andes-card--default ui-search-result ui-search-result--core andes-card--padding-default"})
        for c in search:
            if (item['alt'] == c.contents[0].contents[0]['title']):
                num = float(c.contents[1].find_all("span", {"class": "price-tag-fraction"})[0].contents[0])
    finally:
        sume = sume + num
        if(num<mini):
            mini=num
        if (num>maxi):
            maxi=num
        ## generacion de json
        datos.append({'Nombre': item['alt'],  'Url': price[0]['href'], 'Image': item['data-src'], 'Precio': num})            
average= sume/len(card)
average = round(average,2) 

dicJson = {"Nombre" : NombreArticulo,"Image":Image_articulo,"Vendedor":name_vendedor,"Precio":precio ,"Puntos":calification_points, "Recomendado": recomendado, "Ventas":ventas_completadas,"Time":años_vendiendo,"typeTime":time,"Promedio":average, "Maximo":maxi,"Minimo": mini,'otrosDatos': datos}
dicJson

{'Nombre': 'Camilla Portátil Para Masajes Reforzada Con Maleta',
 'Image': 'https://http2.mlstatic.com/D_NQ_NP_811579-MEC45598429009_042021-F.jpg',
 'Vendedor': 'CUENCA.DISTRIBUIDORA',
 'Precio': '99',
 'Puntos': ['3251', '579', '401'],
 'Recomendado': '88%',
 'Ventas': '26390',
 'Time': '3',
 'typeTime': 'años',
 'Promedio': 104.43,
 'Maximo': 170.0,
 'Minimo': 35.0,
 'otrosDatos': [{'Nombre': 'Camilla Portátil Para Masajes Reforzada Con Maleta',
   'Url': 'https://articulo.mercadolibre.com.ec/MEC-430304751-camilla-portatil-para-masajes-reforzada-con-maleta-_JM#position=1&type=item&tracking_id=14df1d83-6373-429a-86b7-699c45e93b91',
   'Image': 'https://http2.mlstatic.com/D_NQ_NP_811579-MEC45598429009_042021-V.jpg',
   'Precio': 99.0},
  {'Nombre': 'Camilla Portátil Para Masajes Reforzada Con Maleta',
   'Url': 'https://articulo.mercadolibre.com.ec/MEC-430304751-camilla-portatil-para-masajes-reforzada-con-maleta-_JM#position=1&type=item&tracking_id=14df1d83-6373-429a-86b7-699c45e93b91'

In [212]:
datos = soup_other_article.find_all("div", {"class": "ui-search-result__content-wrapper"})
datos[0].contents[1].find_all("span", {"class": "price-tag-fraction"})[0].contents[0]

'99'

In [228]:
datos = soup_other_article.find_all("div", {"class": "andes-card andes-card--flat andes-card--default ui-search-result ui-search-result--core andes-card--padding-default"})
if ('Camilla Portátil Para Masajes Reforzada Con Maleta' == datos[0].contents[0].contents[0]['title']):
    print(datos[0].contents[1].find_all("span", {"class": "price-tag-fraction"})[0].contents[0])


99


### codigo evitable

In [119]:
links = soup_other_article.find_all("a", {"class": "ui-search-link"})
#links

In [111]:
other_link = []
for item in links:
    try:
        if item['title'] in other_name:
            if item['href'] in other_link:
                pass
            else:
                num = int(other_name.index(item['title']))           
                other_link.insert(num, item['href'])
    except:
        pass
len(other_link)

48

In [112]:
other_price = []
prices = soup_other_article.find_all("span", {"class": "price-tag-fraction"})
sume = 0
maxi = 0
mini = 9999999
for price in prices:
    num = int(price.contents[0].replace(".",""))
    sume = sume + num
    if(num<mini):
        mini=num
    if (num>maxi):
        maxi=num    
    other_price.append(num)
average= sume/len(prices)
average = round(average,2)
print (average, maxi, mini)
print(len(other_price))

662.9 1119 89
48


In [115]:
len(other_price)

48

In [116]:
len(other_name)

48

In [117]:
len(other_link)

48

In [118]:
datos = []
for i in range(0,len(other_name)):
    datos.append({'Nombre': other_name[i],  'Url': other_link[i], 'Image': other_img[i], 'Precio': other_price[i]})
datos

[{'Nombre': 'Hp Pavilion X360 14-dh2041 Core 5-1035g1 256gb 8gb Inc. Iva',
  'Url': 'https://articulo.mercadolibre.com.ec/MEC-430292466-hp-pavilion-x360-14-dh2041-core-5-1035g1-256gb-8gb-inc-iva-_JM?searchVariation=80392576392#searchVariation=80392576392&position=1&type=item&tracking_id=d74f5363-474c-457c-904d-4207082141ad',
  'Image': 'https://http2.mlstatic.com/D_NQ_NP_750795-MEC44760931770_012021-W.jpg',
  'Precio': 779},
 {'Nombre': 'Dell Lenovo Hp Intel Core I3 10ma Gen Garantia I5 ',
  'Url': 'https://articulo.mercadolibre.com.ec/MEC-430286913-dell-lenovo-hp-intel-core-i3-10ma-gen-garantia-i5-_JM#position=2&type=item&tracking_id=d74f5363-474c-457c-904d-4207082141ad',
  'Image': 'https://http2.mlstatic.com/D_NQ_NP_664503-MEC41503506143_042020-W.jpg',
  'Precio': 569},
 {'Nombre': 'Laptop Lenovo Ryzen 7 = I7 Nueva Con Garantia Hp Dell',
  'Url': 'https://articulo.mercadolibre.com.ec/MEC-429752127-laptop-lenovo-ryzen-7-i7-nueva-con-garantia-hp-dell-_JM?searchVariation=77106916187#se

## Ebay

In [1]:
import requests
from bs4 import BeautifulSoup

In [39]:
NombreArticulo= 'Laptop hp'
Name = NombreArticulo.split(' ')
Name

['Laptop', 'hp']

In [40]:
URL = "https://www.ebay.com/sch/i.html?_from=R40&_trksid=m570.l1313&_nkw="+NombreArticulo
print(URL)
page_Ebay = requests.get(URL)
soup_Ebay = BeautifulSoup(page_Ebay.content, 'html.parser')

https://www.ebay.com/sch/i.html?_from=R40&_trksid=m570.l1313&_nkw=Laptop hp


In [17]:
soup_Ebay

<!DOCTYPE html>
<!--[if IE 9]><html class="ie9 srp-ds6" lang="es"><![endif]--><!--[if gt IE 9]><!--><html class="srp-ds6" lang="es"><!--<![endif]--><!--M#s0-1--><noscript class="x-page-config" id="s0-1"></noscript><!--M/--><head><meta content="IE=edge" http-equiv="X-UA-Compatible"/><script>"use strict";if(window.PerformanceObserver&&performance&&performance.mark&&performance.getEntriesByName){window.SRP=window.SRP||{};var paintObserver=new window.PerformanceObserver(function(e){var r=e.getEntries();r.sort(function(e,r){return e.startTime-r.startTime});var n=r[1].startTime;window.SRP.TTI_TIMER={lastInteractiveWindow:n};var t=new window.PerformanceObserver(function(e){for(var r=e.getEntries(),i=0,a=r.length;i<a;i++)r[i].startTime-n>=5e3&&(window.SRP.TTI_TIMER.timeToInteract=n,t.disconnect()),n=r[i].startTime+r[i].duration,window.SRP.TTI_TIMER.lastInteractiveWindow=n});t.observe({entryTypes:["longtask"]}),paintObserver.disconnect()});paintObserver.observe({entryTypes:["paint"]})};</script

In [41]:
precios_Ebay = soup_Ebay.find_all("span", {"class": "s-item__price"})

In [42]:
float(precios_Ebay[0].contents[0].replace('USD',''))

184.99

In [43]:
otros_precios = []

sume = 0
maxi = 0
mini = 9999999
for price in precios_Ebay:
    try:
        num = float(price.contents[0].replace('USD',''))
        sume = sume + num
        if(num<mini):
            mini=num
        if (num>maxi):
            maxi=num
        otros_precios.append(num)
    except:
        pass
    
average= sume/len(precios_Ebay)
average = round(average,2)
print (average, maxi, mini)

301.37 800.0 1.0


61

In [54]:
cad= 'USD2 481.36'
cad = cad.replace(' ','')
cad

'USD2481.36'

##### obtener data completa

In [53]:
otras_opciones_img = []
data = soup_Ebay.find_all("img", {"class": "s-item__image-img"})
data[0]

<img alt="HP Chromebook 14 Intel Celeron N3350 4GB Ram 32GB eMMC Pizarra Gris-Intel" class="s-item__image-img" loading="eager" onerror="window.SRP.metrics.imageLoadError.count++; " onload='SITE_SPEED.ATF_TIMER.measure(this); if (performance &amp;&amp; performance.mark) { performance.mark("first-meaningful-paint"); };if(this.width === 80 &amp;&amp; this.height === 80) {window.SRP.metrics.imageEmptyError.count++;}' src="https://i.ebayimg.com/thumbs/images/g/qHgAAOSwYl5fQKdL/s-l225.jpg"/>

In [54]:
for item in data:
    image = item['src']
    otras_opciones_img.append(image)    
#otras_opciones    

In [72]:
otras_opciones_url = []
name_opciones=[]
data = soup_Ebay.find_all("a", {"class": "s-item__link"})
#data[2].find_all("h3")[0].contents[0].contents[0]

In [73]:
for item in data:
    url = item['href']
    otras_opciones_url.append(url)
    name = str(item.find_all("h3")[0].contents[0])
    #print(name)
    char = name.split()    
    if(char[0]!='<span'):
        name_opciones.append(name)
    else:        
        try:
            name = str(item.find_all("h3")[0].contents[1])
        except:
            name = str(item.find_all("h3")[0].contents[0].contents[0])
        name_opciones.append(name) 

In [74]:
len(name_opciones)

61

In [75]:
len(otras_opciones_url)

61

In [76]:
len(otras_opciones_img)

61

In [78]:
len(otros_precios)

61

In [77]:
datos = []
for i in range(0,len(name_opciones)):
    datos.append({'Nombre': name_opciones[i],  'Url': otras_opciones_url[i], 'Image': otras_opciones_img[i], 'Precio': otros_precios[i]})

In [19]:
datos

[{'Nombre': 'TARJETA DE VIDEO DELL INSPIRON 1720 NVIDIA GEFORCE 8600m GT GRAFICA VIDEO',
  'url': 'https://www.ebay.com/itm/TARJETA-DE-VIDEO-DELL-INSPIRON-1720-NVIDIA-GEFORCE-8600m-GT-GRAFICA-VIDEO/154404294267?_trkparms=ispr%3D1&hash=item23f3368e7b:g:AYgAAOSwvEJgbEVz&amdata=enc%3AAQAFAAACgBaobrjLl8XobRIiIML1V4Imu%252Fn%252BzU5L90Z278x5ickkxGz2ccqatkEnoeekHqhLkWc3SglqFmjKtHiJ%252FePriPS3uNGMjRTiPiwQZWfMIBaFCJ7VXKP%252BIe9eyWiDlQw5UfO0ckiWgdvqfGpyeS9a9hkX0sK6uKLKtQ2PHy5T2LSfZhaKiksRHzXg4FkJsgRp%252Fm6u2p131cmwJ%252FBqskl7Igz9a08PrCEpenxWc%252BLhNUNEz4WjpPTDtf6JvzmIfMrFf5kR%252BvD2c9VG8OEiXj4xkrBaYN3Jt%252Fq03hbf6zBIFDYCrtYSftLrC6odLuf8H3xAEY5hJo2n6ve7SicHd5gmd1xEW0O1hgFu4Dbem3wKq%252B5917D9tnfyGDtuTonzOh6LlPZXTpgCD5jZxCjcFw65Umj9GhWeBK2o80EFAmBtO5ePIN3wEPKzPqBa%252BvFwu63awfJ270tlQZzQR0zGI5SPYBaI%252BMoDtTCB8frTNbR%252FBPO%252Bl0f66P90lCQUOLh7kaQQCsGTYWzM3GmxjLadT%252FhjwUaHTWILXkyOFlixNDOADcxvFJLL9EOZYkYaEfd1JE%252FoAhlISIVlRcNGxOPr2%252FiKqV1wk04qj%252F6ctBvRktFAKeD5hBiDqSde%252FzuKyj

#### OLX

In [1]:
articulo = 'laptop hp'
URL = "https://www.olx.com.ec/items/q-"+ articulo   
URL

'https://www.olx.com.ec/items/q-laptop hp'

In [3]:
page_Olx = requests.get(URL)
soup_Olx = BeautifulSoup(page_Olx.content, 'html.parser')

In [16]:
card = soup_Olx.find_all("li", {"data-aut-id": "itemBox"}) 
url = card[0].contents[0]['href']
url

'/item/teclado-gamer-hp-iluminado-k110-qwerty-usb-gaming-iid-1101563516'

In [58]:
sume = 0
maxi = 0
mini = 9999999

datos = []
for item in card:
    values= item.contents[0].find_all("img")[0]
    num = float(item.contents[0].find_all("span", {"data-aut-id": "itemPrice"})[0].contents[0].replace(' ','').replace('$','').replace(',',''))    
    
    sume = sume + num
    if(num<mini):
        mini=num
    if (num>maxi):
        maxi=num

    ## generacion de json
    datos.append({'Nombre': values['alt'],  'Url': item.contents[0]['href'], 'Image': values['src'], 'Precio': num})
    
average= sume/len(card)
average = round(average,2) 

dicJson = dicJson = {"Promedio":average, "Maximo":maxi,"Minimo": mini, "otrosDatos":datos}
dicJson
    

{'Promedio': 501.7,
 'Maximo': 1500.0,
 'Minimo': 1.0,
 'otrosDatos': [{'Nombre': 'Teclado Gamer Hp Iluminado K110 Qwerty Usb Gaming',
   'Url': '/item/teclado-gamer-hp-iluminado-k110-qwerty-usb-gaming-iid-1101563516',
   'Image': 'https://apollo-virginia.akamaized.net:443/v1/files/0iehlmfhgyl-EC/image;s=272x0',
   'Precio': 20.0},
  {'Nombre': 'hp omen 15',
   'Url': '/item/hp-omen-15-iid-1104044854',
   'Image': 'https://apollo-virginia.akamaized.net:443/v1/files/13tzw9pgf4vt2-EC/image;s=272x0',
   'Precio': 1366.0},
  {'Nombre': 'COMPUTADORA HP 8GB RAM Y 1TB DISCO DURO  SEMINUEVAS',
   'Url': '/item/computadora-hp-8gb-ram-y-1tb-disco-duro-seminuevas-iid-1104682989',
   'Image': 'https://apollo-virginia.akamaized.net:443/v1/files/8rf3qs11egep-EC/image;s=272x0',
   'Precio': 305.0},
  {'Nombre': 'COMPUTADOR COMPLETO Core i5 6ta GENERACION 8GB RAM y 500 GB DISCO DURO HP 800 G2 TECLADO Y MOUSE420',
   'Url': '/item/computador-completo-core-i5-6ta-generacion-8gb-ram-y-500-gb-disco-duro-h

In [31]:
values= card[3].contents[0].find_all("img")[0]
print(values['alt'])
print(values['src'])
values

COMPUTADOR COMPLETO Core i5 6ta GENERACION 8GB RAM y 500 GB DISCO DURO HP 800 G2 TECLADO Y MOUSE420
https://apollo-virginia.akamaized.net:443/v1/files/ssnoes3pjhog1-EC/image;s=272x0


<img alt="COMPUTADOR COMPLETO Core i5 6ta GENERACION 8GB RAM y 500 GB DISCO DURO HP 800 G2 TECLADO Y MOUSE420" class="_3Kg_w" sizes="(min-width: 1280px) 15vw, (min-width: 1024px) 20vw, (min-width: 960px) 25vw, (min-width: 540px) 25vw, (min-width: 320px) 30vw, 35vw" src="https://apollo-virginia.akamaized.net:443/v1/files/ssnoes3pjhog1-EC/image;s=272x0" srcset="https://apollo-virginia.akamaized.net:443/v1/files/ssnoes3pjhog1-EC/image;s=100x200;q=60 100w,
                    https://apollo-virginia.akamaized.net:443/v1/files/ssnoes3pjhog1-EC/image;s=200x400;q=60 200w,
                    https://apollo-virginia.akamaized.net:443/v1/files/ssnoes3pjhog1-EC/image;s=300x600;q=60 300w,
                    https://apollo-virginia.akamaized.net:443/v1/files/ssnoes3pjhog1-EC/image;s=400x800;q=60 400w,
                    https://apollo-virginia.akamaized.net:443/v1/files/ssnoes3pjhog1-EC/image;s=600x1200;q=60 600w"/>

In [37]:
card[3].contents[0]

<a class="fhlkh" href="/item/computador-completo-core-i5-6ta-generacion-8gb-ram-y-500-gb-disco-duro-hp-800-g2-teclado-y-mouse420-iid-1104728255"><figure class="_2grx4" data-aut-id="itemImage"><div class="_1czk1"><div class="_1VzGN _1VX6L"></div></div><noscript><img alt="COMPUTADOR COMPLETO Core i5 6ta GENERACION 8GB RAM y 500 GB DISCO DURO HP 800 G2 TECLADO Y MOUSE420" class="_3Kg_w" sizes="(min-width: 1280px) 15vw, (min-width: 1024px) 20vw, (min-width: 960px) 25vw, (min-width: 540px) 25vw, (min-width: 320px) 30vw, 35vw" src="https://apollo-virginia.akamaized.net:443/v1/files/ssnoes3pjhog1-EC/image;s=272x0" srcset="https://apollo-virginia.akamaized.net:443/v1/files/ssnoes3pjhog1-EC/image;s=100x200;q=60 100w,
                    https://apollo-virginia.akamaized.net:443/v1/files/ssnoes3pjhog1-EC/image;s=200x400;q=60 200w,
                    https://apollo-virginia.akamaized.net:443/v1/files/ssnoes3pjhog1-EC/image;s=300x600;q=60 300w,
                    https://apollo-virginia.akamaize

In [52]:
precio = card[3].contents[0].find_all("span", {"data-aut-id": "itemPrice"})[0].contents[0].replace(' ','').replace('$','')
precio

'420'

In [6]:
precios = soup_Olx.find_all("span", {"data-aut-id": "itemPrice"})
precios

[<span class="_89yzn" data-aut-id="itemPrice">$ 20</span>,
 <span class="_89yzn" data-aut-id="itemPrice">$ 1,366</span>,
 <span class="_89yzn" data-aut-id="itemPrice">$ 305</span>,
 <span class="_89yzn" data-aut-id="itemPrice">$ 420</span>,
 <span class="_89yzn" data-aut-id="itemPrice">$ 649</span>,
 <span class="_89yzn" data-aut-id="itemPrice">$ 30</span>,
 <span class="_89yzn" data-aut-id="itemPrice">$ 655</span>,
 <span class="_89yzn" data-aut-id="itemPrice">$ 855</span>,
 <span class="_89yzn" data-aut-id="itemPrice">$ 1,500</span>,
 <span class="_89yzn" data-aut-id="itemPrice">$ 280</span>,
 <span class="_89yzn" data-aut-id="itemPrice">$ 1</span>,
 <span class="_89yzn" data-aut-id="itemPrice">$ 228</span>,
 <span class="_89yzn" data-aut-id="itemPrice">$ 420</span>,
 <span class="_89yzn" data-aut-id="itemPrice">$ 50</span>,
 <span class="_89yzn" data-aut-id="itemPrice">$ 460</span>,
 <span class="_89yzn" data-aut-id="itemPrice">$ 920</span>,
 <span class="_89yzn" data-aut-id="itemPr

In [5]:
if(len(precios) == 0):
    dicJson = {}
    
try:       
    sume = 0
    maxi = 0
    mini = 9999999

    for price in precios:
        num = float(price.contents[0].replace('$',''))
        sume = sume + num
        if(num<mini):
            mini=num
        if (num>maxi):
            maxi=num
    if (len(precios) != 0):
        average= sume/len(precios)
        average= round(average,2)
    else:
        average = 0
except :
    average = 0
    maxi = 0
    mini = 0

print(average,mini,maxi)

0 0 0


In [ ]:
##### otros datos

In [90]:
otras_img = []
otras_name = []
otras_url = []
imgns = soup_Olx.find_all("img", {"class": "_1N079"})
for item in imgns:    
    otras_img.append(item['src'])
    otras_name.append(item['alt'])

In [91]:
imgns = soup_Olx.find_all("img", {"class": "_3Kg_w"})
for item in imgns:
    otras_img.append(item['src'])
    otras_name.append(item['alt'])

In [92]:
len(otras_img)

20

In [93]:
len(otras_name)

20

In [97]:
name = soup_Olx.find_all("a")
len(name)

134